In [1]:
## TIME POLLING ##
import time
import datetime
import pandas as pd
import numpy as np
import csv
import sys
from settings import API_KEY, API_SECRET, API_BASE
from lib.bitmex import BitMEX
from lib import bitmex
import BK_FUNCTIONS
BK_FUNCTIONS.BeginShort(BK_FUNCTIONS.GetOrderBookPrice, BK_FUNCTIONS.GetBalance)
BK_FUNCTIONS.EndShortMkt()